Now that we have all of the WHIM data as shown in WHIM_plot, let's take those functions and make plots for the different mass definitions in the catalog.

In [8]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import row, column, gridplot
from bokeh.models import BoxSelectTool, LassoSelectTool, Spacer
from bokeh.plotting import figure
from sklearn.linear_model import LinearRegression
output_notebook()

Loading BokehJS ...

In [3]:
# Load in dask 4096 set (no nans!)
d = np.loadtxt('./4096z05/da_WHIM_data.txt')
ID = d[:,0]
hm = d[:,1]
r = d[:,2]
WHIM = d[:,3::]

# Also load in other mass defs
# IDs, i, j, k, m, r, m200, r200, m180, r180
IDcat, hm200, r200, hm180, r180 = np.loadtxt('./catalogs/catalog_iso138_200_180_trimmed.txt',usecols=(0,6,7,8,9),unpack=True)
# Importantly, r200 and r180 are in pixel units whereas r and WHIM are in Mpc/h. Let's fix this:
l, size, z = np.loadtxt('./4096z05/universe_data.txt')
r200 *= size/l
r180 *= size/l

In [30]:
# Check IDs are in the same order:
print(np.nonzero(ID-IDcat))
# Check to make sure there are no nans:
print(WHIM[np.isnan(WHIM)])

(array([], dtype=int64),)
[]


In [4]:
# Scale the WHIM:
WHIM_scaled = np.copy(WHIM)
WHIM200_scaled = np.copy(WHIM)
WHIM180_scaled = np.copy(WHIM)
for i in range(len(d)):
    WHIM_scaled[i,:] /= r[i]
    WHIM200_scaled[i,:] /= r200[i]
    WHIM180_scaled[i,:] /= r180[i]

In [18]:
# Define useful tools:
TOOLS='hover,crosshair,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,pan'

# Tools I'm not using: 
#lasso_select - Doesn't work v well for hist.
#poly_select - Overkill

# Set some std colors:
line = 'black'
bck = 'white'

In [15]:
# First plot histogram of mass distribution
def hm_hist(hm,nbins=10):
    '''
    Takes in a set of halo masses and makes the histogram.
    nbins = 10 by default.
    '''
    
    # Take log of halo masses:
    log_hm = np.log10(hm)

    # Get bin positions
    hist, edges = np.histogram(log_hm, density=False, bins=nbins)

    p = figure(title='',tools=TOOLS, background_fill_color=bck,x_range=(min(edges),max(edges)),y_range=(0,1.1*max(hist)))
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color=line, fill_color='steelblue')
    
    # Axis labels
    p.xaxis.axis_label='log10(Mass/Solar Mass)'
    p.xaxis.axis_label_text_font_style='normal'
    p.yaxis.axis_label='Count'
    p.yaxis.axis_label_text_font_style='normal'
    
    show(p)

In [39]:
# First make a scatter plot with hists around it, and fit a linear regression line.
# It would be very cool to turn this into this, but time!
# https://github.com/bokeh/bokeh/blob/master/examples/app/selection_histogram.py
# Also want to include more advanced fitting (ie. test and training set), but time.
def scatter(hm,WHIM_sizes,y_units='(Mpc/h)',fit=False):
    '''
    Takes in a halo masses with their corresponding WHIM_sizes. 
    Make sure the indices are the same between the sets.
    Produces a scatter plot, and will fit with linear regression if fit=True.
    Can input own y_units
    '''
    # Manipulate data:
    x = np.transpose(np.log10([hm]*26)).flatten()
    y = WHIM_sizes.flatten()
    
    # Central figure:
    p = figure(tools=TOOLS+',lasso_select',background_fill_color=bck, min_border=10, min_border_left=50,
               toolbar_location='above', x_range=(min(x)-0.01,max(x)+0.01),y_range=(0,max(y[~np.isnan(y)])),
               x_axis_location=None, y_axis_location=None)
    r = p.scatter(x=x, y=y,size=1,color='steelblue')
    p.select(BoxSelectTool).select_every_mousemove = False
    p.select(LassoSelectTool).select_every_mousemove = False
    
    # Put in several args for line at once
    LINE_ARGS = dict(color=line, line_color=None)
    
    # Create the horizontal histogram
    hhist, hedges = np.histogram(x, bins=20,range=(min(x)-0.01,max(x)+0.01))
    hzeros = np.zeros(len(hedges)-1)
    hmax = max(hhist)*1.1
    
    ph = figure(toolbar_location=None, plot_width=p.plot_width, plot_height=200, x_range=p.x_range,
                y_range=(-hmax, hmax), min_border=10, min_border_left=50, y_axis_location="right")
    ph.xgrid.grid_line_color = None
    ph.yaxis.major_label_orientation = np.pi/4
    ph.background_fill_color = bck

    ph.quad(bottom=0, left=hedges[:-1], right=hedges[1:], top=hhist, color='lightsteelblue', line_color=line)
    hh1 = ph.quad(bottom=0, left=hedges[:-1], right=hedges[1:], top=hzeros, alpha=0.5, **LINE_ARGS)
    hh2 = ph.quad(bottom=0, left=hedges[:-1], right=hedges[1:], top=hzeros, alpha=0.1, **LINE_ARGS)

    # Create the vertical histogram
    vhist, vedges = np.histogram(y, bins=20,range=(min(y),max(y[~np.isnan(y)])))
    vzeros = np.zeros(len(vedges)-1)
    vmax = max(vhist)*1.1

    pv = figure(toolbar_location=None, plot_width=200, plot_height=p.plot_height, x_range=(-vmax, vmax),
                y_range=p.y_range, min_border=10, y_axis_location="right")
    pv.ygrid.grid_line_color = None
    pv.xaxis.major_label_orientation = np.pi/4
    pv.background_fill_color = bck

    pv.quad(left=0, bottom=vedges[:-1], top=vedges[1:], right=vhist, color='lightsteelblue', line_color=line)
    vh1 = pv.quad(left=0, bottom=vedges[:-1], top=vedges[1:], right=vzeros, alpha=0.5, **LINE_ARGS)
    vh2 = pv.quad(left=0, bottom=vedges[:-1], top=vedges[1:], right=vzeros, alpha=0.1, **LINE_ARGS)

    layout = column(row(p, pv), row(ph,Spacer(width=200, height=200)))
    
    # Fit with Linear Regression
    if fit:
        model = LinearRegression(fit_intercept=True)
        model.fit(x[:, np.newaxis], y)
        print("R^2 = {:.3f}".format(model.score(x[:, np.newaxis], y)))

        xfit = np.linspace(min(x)-0.01, max(x)+0.01, 1000)
        yfit = model.predict(xfit[:, np.newaxis])

        p.line(xfit, yfit,line_color='coral',line_width=2,line_alpha=0.5)
    
    # Axis labels
    ph.xaxis.axis_label='log10(Mass/Solar Mass)'
    ph.xaxis.axis_label_text_font_style='normal'
    pv.yaxis.axis_label='WHIM Size '+y_units
    pv.yaxis.axis_label_text_font_style='normal'

    
    show(layout)
    
    if False:
        # Define the function that will happen when we select the data, ie. the side histograms will update.
        def update(attr, old, new):
            inds = np.array(new['1d']['indices'])
            if len(inds) == 0 or len(inds) == len(x):
                hhist1, hhist2 = hzeros, hzeros
                vhist1, vhist2 = vzeros, vzeros
            else:
                neg_inds = np.ones_like(x, dtype=np.bool)
                neg_inds[inds] = False
                hhist1, _ = np.histogram(x[inds], bins=hedges)
                vhist1, _ = np.histogram(y[inds], bins=vedges)
                hhist2, _ = np.histogram(x[neg_inds], bins=hedges)
                vhist2, _ = np.histogram(y[neg_inds], bins=vedges)

            hh1.data_source.data["top"]   =  hhist1
            hh2.data_source.data["top"]   = -hhist2
            vh1.data_source.data["right"] =  vhist1
            vh2.data_source.data["right"] = -vhist2

        r.data_source.on_change('selected', update)

In [25]:
# Now make a box plot function:
def boxplot(hm,WHIM_sizes,y_units='(Mpc/h)',nbins=10):
    '''
    Takes in a halo masses with their corresponding WHIM_sizes. 
    Make sure the indices are the same between the sets.
    Will produce a boxplot with quartiles, median, and whiskers extending to 0.05 and 0.95.
    nbins is an optional argument that is by default set to 10.
    '''
    # Take log:
    log_hm = np.log10(hm)
    
    # Get the histogram data for log_hm:
    hist, edges = np.histogram(log_hm, density=False, bins=nbins)
    
    # List of indices for each bin
    ind_list = [np.argwhere((log_hm < edges[i+1]) & (log_hm >= edges[i])) for i in range(nbins)]

    # Get position for bins and lines (weighted bin position)
    pos_bin = [(edges[i+1]+edges[i])/2 for i in range(nbins)]
    pos_line = [log_hm[ind_list[i]].mean() if ind_list[i].size else pos_bin[i] for i in range(nbins)]

    # Add the WHIM_data from the correct spot.
    WHIMdata = [WHIM_sizes[ind_list[i]].flatten() for i in range(nbins)]
    
    # Also remove nans:
    for i in range(len(WHIMdata)):
        WHIMdata[i] = WHIMdata[i][~np.isnan(WHIMdata[i])]

    # Set up quartiles. If there is no data then set to zero.
    q1 = [(np.percentile(WHIM,25) if WHIM.size else 0) for WHIM in WHIMdata]
    q2 = [(np.percentile(WHIM,50) if WHIM.size else 0) for WHIM in WHIMdata]
    q3 = [(np.percentile(WHIM,75) if WHIM.size else 0) for WHIM in WHIMdata]
    upper = [(np.percentile(WHIM,95) if WHIM.size else 0) for WHIM in WHIMdata]
    lower = [(np.percentile(WHIM,5) if WHIM.size else 0) for WHIM in WHIMdata]

    p = figure(tools=TOOLS, background_fill_color=bck, title="", x_range=(min(edges),max(edges)), y_range=(0,1.1*max(upper)))

    # stems
    p.segment(pos_line, upper, pos_line, q3, line_color=line)
    p.segment(pos_line, lower, pos_line, q1, line_color=line)

    # boxes
    p.vbar(pos_bin, [edges[i+1]-edges[i] for i in range(nbins)], q2, q3, fill_color='steelblue', line_color=line)
    p.vbar(pos_bin, [edges[i+1]-edges[i] for i in range(nbins)], q1, q2, fill_color='coral', line_color=line)

    # whiskers (almost-0 height rects simpler than segments)
    p.rect(pos_line, lower, [(edges[i+1]-edges[i])/2 for i in range(nbins)], 0.001, fill_color=line,line_color=line)
    p.rect(pos_line, upper, [(edges[i+1]-edges[i])/2 for i in range(nbins)], 0.001, fill_color=line,line_color=line)
    
    # Axis labels
    p.xaxis.axis_label='log10(Mass/Solar Mass)'
    p.xaxis.axis_label_text_font_style='normal'
    p.yaxis.axis_label='WHIM Size '+y_units
    p.yaxis.axis_label_text_font_style='normal'

    show(p)

In [16]:
hm_hist(hm)

In [17]:
hm_hist(hm200)

In [19]:
hm_hist(hm180)

In [40]:
scatter(hm,WHIM_scaled,y_units='(iso radius)',fit=True)

R^2 = 0.025


In [41]:
scatter(hm200,WHIM200_scaled,y_units='(r200)',fit=True)

R^2 = 0.024


In [42]:
scatter(hm180,WHIM180_scaled,y_units='(r180)',fit=True)

R^2 = 0.025


In [26]:
boxplot(hm,WHIM_scaled,y_units='(iso radius)')

In [27]:
boxplot(hm200,WHIM200_scaled,y_units='(r200)')

In [28]:
boxplot(hm180,WHIM180_scaled,y_units='(r180)')